In [5]:
from dotenv import load_dotenv
from pathlib import Path

dotenv_path = Path('.') / '.env'  
load_dotenv(dotenv_path=dotenv_path)


True

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()  # ✅ This loads the .env file into the environment

# ✅ Now load the variable using the exact name from the .env file
fsquare_api_key = os.getenv("FOURSQUARE_API_KEY")
yelp_api_key = os.getenv("YELP_API_KEY")



In [9]:

import requests
import pandas as pd
import time

df_stations = pd.read_csv("bike_stations.csv")

fsquare_api_key = "fsq3mY8gA3kNr3ohupZu2Q24lP6dQWjWE7gDtfcipnZRMsA="

headers = {
    "Accept": "application/json",
    "Authorization": fsquare_api_key
}

params = {
    "ll": "24.451202,54.334510",  # lat,long
    "radius": 1000,
    "categories": "13065",  # restaurants
    "limit": 10
}

url = "https://api.foursquare.com/v3/places/search"
response = requests.get(url, headers=headers, params=params)
results = response.json()
#print(results["results"][0])
#results

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [10]:
df_stations.head()

,network,name,latitude,longitude,free_bikes,empty_slots,slots,calculated_total
0,bixi_montreal,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,7,12,19,19
1,bixi_montreal,Square-Victoria (Viger / du Square-Victoria),45.502085,-73.562941,14,22,37,36
2,bixi_montreal,Hudson / Goyer,45.507437,-73.632658,11,12,23,23
3,bixi_montreal,Marché Jean-Talon (Casgrain / Shamrock),45.535190,-73.615482,9,17,29,26
4,bixi_montreal,du Fort Saint-Louis / de la Perrière,45.613330,-73.451540,8,7,15,15


In [11]:
df_stations = pd.read_csv("bike_stations.csv")
poi_results = []
for index, row in df_stations.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    station_name = row["name"]

    params = {
        "ll": f"{lat},{lon}",
        "radius": 1000,
        "categories": "13065",  # Restaurants
        "limit": 10
    }

    url = "https://api.foursquare.com/v3/places/search"

    try:
        response = requests.get(url, headers=headers, params=params)
        results = response.json()

        for place in results.get("results", []):
            
            poi_results.append({
                "station": station_name,
                "station_lat": lat,
                "station_lon": lon,
                "place_name": place["name"],
                "category": place["categories"][0]["name"] if place["categories"] else "Unknown",
                "distance": place.get("distance", None),
                "poi_latitude": place.get("geocodes", {}).get("main", {}).get("latitude"),
                "poi_longitude": place.get("geocodes", {}).get("main", {}).get("longitude")
            })

        time.sleep(0.2)  # Respect rate limit
    except Exception as e:
        print(f"Failed for station {station_name}: {e}")

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [69]:
#poi_results
#results

Put your parsed results into a DataFrame

In [12]:
df_fsq = pd.DataFrame(poi_results)
df_fsq.head()
#df_fsq



,station,station_lat,station_lon,place_name,category,distance,poi_latitude,poi_longitude
0,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Benelux,Beer Bar,97,45.463481,-73.566793
1,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Station W,Café,298,45.465472,-73.566614
2,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,BOSSA Prêt-à-manger,Deli,273,45.460468,-73.566939
3,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Maison de Thé Cha Noir,Tea Room,538,45.458128,-73.567611
4,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Bar Palco,Lounge,121,45.463530,-73.567133


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [13]:
yelp_api_key = "QniIwUBLi6vsoSqiBCox6Yjj_BtYDjhY8T6w9B-vn7Mft62fG_2cMCcd9hcEBjcfJFp9ByKy6oiNff5UHZx1QBV-20FEh6BYmi6Gt2m5uiiNb8vxGacLzxUp16EjaHYx"

headers = {
    "Authorization": f"Bearer {yelp_api_key}"
    #'Authorization: Bearer yelp_api_key'
}

params = {
    "term": "restaurants",           
    "latitude": 45.462831,
    "longitude": -73.565938,
    "radius": 1000,                  
    "limit": 10                      
}

url = "https://api.yelp.com/v3/businesses/search"
response = requests.get(url, headers=headers, params=params)
yelp_results = response.json()

if "businesses" in yelp_results and len(yelp_results["businesses"]) > 0:
    print(yelp_results["businesses"][0])
else:
    print("No businesses found.")

#yelp_results




{'id': 'T4Vg3C9_cShGJL4Hq0Yahw', 'alias': 'les-street-monkeys-montréal', 'name': 'Les Street Monkeys', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/_QKfE3Avveh9_eE37fMYEA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/les-street-monkeys-montr%C3%A9al?adjust_creative=1IU9I5gzXj4oLPILbrOiEg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1IU9I5gzXj4oLPILbrOiEg', 'review_count': 83, 'categories': [{'alias': 'cambodian', 'title': 'Cambodian'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}], 'rating': 4.4, 'coordinates': {'latitude': 45.46767, 'longitude': -73.56688}, 'transactions': [], 'price': '$$', 'location': {'address1': '3625 Rue Wellington', 'address2': '', 'address3': None, 'city': 'Montreal', 'zip_code': 'H4G 1T9', 'country': 'CA', 'state': 'QC', 'display_address': ['3625 Rue Wellington', 'Montreal, QC H4G 1T9', 'Canada']}, 'phone': '+15147681818', 'display_phone': '+1 514-768-1818', 'distance': 546.9993357634223, 'business_hours'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [14]:

# Replace with your real Yelp API key
yelp_api_key = "QniIwUBLi6vsoSqiBCox6Yjj_BtYDjhY8T6w9B-vn7Mft62fG_2cMCcd9hcEBjcfJFp9ByKy6oiNff5UHZx1QBV-20FEh6BYmi6Gt2m5uiiNb8vxGacLzxUp16EjaHYx"
headers = {
    "Authorization": f"Bearer {yelp_api_key}"
}

# List to hold all POI results
yelp_poi_results = []

# Loop through each station
for index, row in df_stations.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    station_name = row["name"]

    params = {
        "term": "restaurant",
        "latitude": lat,
        "longitude": lon,
        "radius": 5000,     
        "limit": 10
    }

    url = "https://api.yelp.com/v3/businesses/search"

    try:
        response = requests.get(url, headers=headers, params=params)
        yelp_data = response.json()

        if "businesses" in yelp_data and len(yelp_data["businesses"]) > 0:
            for biz in yelp_data["businesses"]:
                yelp_poi_results.append({
                    "station": station_name,
                    "station_lat": lat,
                    "station_lon": lon,
                    "business_name": biz.get("name", "N/A"),
                    "category": biz["categories"][0]["title"] if biz.get("categories") else "Unknown",
                    "rating": biz.get("rating", None),
                    "review_count": biz.get("review_count", 0),
                    "distance_m": biz.get("distance", None),
                    "biz_lat": biz["coordinates"]["latitude"],
                    "biz_lon": biz["coordinates"]["longitude"],
                    "address": biz["location"].get("address1", "Unknown")
                })

        else:
            print(f"No businesses found near: {station_name}")

        time.sleep(0.2)  # Respect Yelp's rate limits

    except Exception as e:
        print(f"Failed for station {station_name}: {e}")


Put your parsed results into a DataFrame

In [15]:
#yelp_poi_results
#yelp_data
yelp_df = pd.DataFrame(yelp_poi_results)
yelp_df.head()

,station,station_lat,station_lon,business_name,category,rating,review_count,distance_m,biz_lat,biz_lon,address
0,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Les Street Monkeys,Cambodian,4.4,83,546.979725,45.467670,-73.566880,3625 Rue Wellington
1,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Joe Beef,Seafood,4.3,849,2365.240370,45.483062,-73.575308,2491 Rue Notre-Dame Ouest
2,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Monarque,Brasseries,4.5,235,4347.922392,45.501580,-73.559620,406 Rue Saint-Jacques
3,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Beba,Argentine,4.7,31,269.706722,45.464869,-73.567814,3900 Rue Éthel
4,Métro de l'Église (Ross / de l'Église),45.462831,-73.565938,Bistro Entre Ciel et Terre,French,4.1,34,553.605610,45.462962,-73.573034,750 Rue de l'Eglise


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided the most data.
While Foursquare provided information on just the location of the various restaurants in and around 5000 m radius of our longitude and latitude points of interest, Yelp provides alot more - category, review count, rating, address and more.

Get the top 10 restaurants according to their rating

In [23]:
# Sorts the Yelp DataFrame by rating (descending), then show top 10
top_10_restaurants = yelp_df.sort_values(by="rating", ascending=False).head(10)

top_10_restaurants[["business_name", "rating", "review_count", "address", "station"]]


,business_name,rating,review_count,address,station
7923,Chez Cesar,5.0,6,3477 Montée Saint Hubert,Gare Longueuil-Saint-Hubert (de l'Aéroport)
6660,Pandora Supper Club,5.0,1,579 Boulevard Saint-Martin Ouest,Sun Valley / de Sillery
3591,Porto Poulet,5.0,2,1612 Rue Fleury E,de St-Firmin / Fleury
5197,Caifan,5.0,8,4542 Rue Saint-Denis,de St-Firmin / Sauvé
8231,Le Garde Manger Italien,5.0,3,6132 Avenue de Monkland,U. Concordia - Campus Loyola (Sherbrooke / Wes...
6211,Le Garde Manger Italien,5.0,3,6132 Avenue de Monkland,Parc Loyola (Doherty / Somerled)
5200,Kaokao Beer Garden,5.0,1,3200 Bd Curé-Labelle,Gare Ste-Dorothée (Hector-Nadon / du Chemin-de...
5202,Macallan’s Pub,5.0,1,2362 Hymus Boulevard,Gare Ste-Dorothée (Hector-Nadon / du Chemin-de...
5204,Le Géraldine,5.0,6,163 Rue Saint-Eustache,Gare Ste-Dorothée (Hector-Nadon / du Chemin-de...
2947,Caifan,5.0,8,4542 Rue Saint-Denis,Berri / Laurier


In [24]:
top_10_restaurants = yelp_df[yelp_df["review_count"] > 5] \
    .sort_values(by="rating", ascending=False) \
    .head(10)

top_10_restaurants.head(10)

,station,station_lat,station_lon,business_name,category,rating,review_count,distance_m,biz_lat,biz_lon,address
6008,de Gaspé / Jarry,45.540780,-73.630401,Caifan,Mexican,5.0,8,4075.061162,45.524179,-73.583853,4542 Rue Saint-Denis
5997,Gilford / St-Denis,45.524976,-73.585589,Caifan,Mexican,5.0,8,167.730579,45.524179,-73.583853,4542 Rue Saint-Denis
9098,Berri / Cherrier,45.519266,-73.569297,Caifan,Mexican,5.0,8,1252.150139,45.524179,-73.583853,4542 Rue Saint-Denis
7705,Jeanne-Mance / Beaubien,45.527650,-73.612300,Caifan,Mexican,5.0,8,2256.928052,45.524179,-73.583853,4542 Rue Saint-Denis
2285,Mozart / Drolet,45.536325,-73.612862,Caifan,Mexican,5.0,8,2638.669477,45.524179,-73.583853,4542 Rue Saint-Denis
3643,Parc de Turin (de Lanaudière / Jean-Talon),45.545350,-73.610330,Caifan,Mexican,5.0,8,3134.028964,45.524179,-73.583853,4542 Rue Saint-Denis
8279,Métro Mont-Royal (Utilités publiques / Rivard),45.524353,-73.581432,Caifan,Mexican,5.0,8,181.699059,45.524179,-73.583853,4542 Rue Saint-Denis
9136,Aylmer / Prince-Arthur,45.508567,-73.577698,Caifan,Mexican,5.0,8,1799.975458,45.524179,-73.583853,4542 Rue Saint-Denis
2758,de Bordeaux / de Bellechasse,45.543259,-73.591546,Caifan,Mexican,5.0,8,2205.726490,45.524179,-73.583853,4542 Rue Saint-Denis
378,Boyer / Rosemont,45.534099,-73.595535,Caifan,Mexican,5.0,8,1434.089051,45.524179,-73.583853,4542 Rue Saint-Denis



Converting all dataframes to csv

In [26]:
df_stations.to_csv("bike_stations.csv", index=False)
yelp_df.to_csv("yelp_pois.csv", index=False)
df_fsq.to_csv("fsq_pois.csv", index=False)